# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7f54c8b0a0>
├── 'a' --> tensor([[-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569]])
└── 'x' --> <FastTreeValue 0x7f7f54c8bdf0>
    └── 'c' --> tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                        [ 0.9263, -1.2560,  1.8936,  0.3023],
                        [ 0.4243, -0.7446,  0.6436, -2.5038]])

In [4]:
t.a

tensor([[-0.1679,  0.4615,  0.0652],
        [-0.4972, -0.2046,  1.0569]])

In [5]:
%timeit t.a

60.1 ns ± 0.064 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7f54c8b0a0>
├── 'a' --> tensor([[-1.1839,  0.5539, -0.5574],
│                   [-0.7222, -0.1995,  0.5073]])
└── 'x' --> <FastTreeValue 0x7f7f54c8bdf0>
    └── 'c' --> tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                        [ 0.9263, -1.2560,  1.8936,  0.3023],
                        [ 0.4243, -0.7446,  0.6436, -2.5038]])

In [7]:
%timeit t.a = new_value

58 ns ± 0.0318 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569]]),
    x: Batch(
           c: tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                      [ 0.9263, -1.2560,  1.8936,  0.3023],
                      [ 0.4243, -0.7446,  0.6436, -2.5038]]),
       ),
)

In [10]:
b.a

tensor([[-0.1679,  0.4615,  0.0652],
        [-0.4972, -0.2046,  1.0569]])

In [11]:
%timeit b.a

56.4 ns ± 0.0411 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2983,  0.9191,  0.4214],
               [ 0.1329,  0.8208, -1.1177]]),
    x: Batch(
           c: tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                      [ 0.9263, -1.2560,  1.8936,  0.3023],
                      [ 0.4243, -0.7446,  0.6436, -2.5038]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.066 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 4.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 55.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7ea6b193d0>
├── 'a' --> tensor([[[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]],
│           
│                   [[-0.1679,  0.4615,  0.0652],
│                    [-0.4972, -0.2046,  1.0569]]])
└── 'x' --> <FastTreeValue 0x7f7ea6b19b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7f54c17340>
├── 'a' --> tensor([[-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569],
│                   [-0.1679,  0.4615,  0.0652],
│                   [-0.4972, -0.2046,  1.0569]])
└── 'x' --> <FastTreeValue 0x7f7f54c17a30>
    └── 'c' --> tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                        [ 0.9263, -1.2560,  1.8936,  0.3023],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 73.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.1 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]],
       
               [[-0.1679,  0.4615,  0.0652],
                [-0.4972, -0.2046,  1.0569]]]),
    x: Batch(
           c: tensor([[[ 1.7278,  0.3037,  0.9278, -0.3101],
                       [ 0.9263, -1.2560,  1.8936,  0.3023],
                       [ 0.4243, -0.7446,  0.6436, -2.5038]],
         

In [26]:
%timeit Batch.stack(batches)

78 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569],
               [-0.1679,  0.4615,  0.0652],
               [-0.4972, -0.2046,  1.0569]]),
    x: Batch(
           c: tensor([[ 1.7278,  0.3037,  0.9278, -0.3101],
                      [ 0.9263, -1.2560,  1.8936,  0.3023],
                      [ 0.4243, -0.7446,  0.6436, -2.5038],
                      [ 1.7278,  0.3037,  0.9278, -0.3101],
                      [ 0.9263, -

In [28]:
%timeit Batch.cat(batches)

140 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 889 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
